## Testing Notebook


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, patterning
from fibsem.structures import BeamType
from pprint import pprint
import matplotlib.pyplot as plt


microscope, settings = utils.setup_session()

In [ ]:
## apply configuration

microscope.apply_configuration()


electron = microscope.get_beam_system_settings(beam_type=BeamType.ELECTRON)
pprint(electron.to_dict())

ion = microscope.get_beam_system_settings(beam_type=BeamType.ION)
pprint(ion.to_dict())



In [ ]:
# flat to beams
microscope.move_flat_to_beam(beam_type=BeamType.ELECTRON)

In [ ]:
microscope.move_flat_to_beam(beam_type=BeamType.ION)

In [ ]:
state = microscope.get_microscope_state()

pprint(state.to_dict())

In [ ]:
# move to state
microscope.set_microscope_state(state)

In [ ]:
## manipulator

microscope.insert_manipulator("PARK")


settings.image.hfw = 400e-6
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")

plt.show()


In [ ]:

from autolamella.workflows import actions
actions.move_needle_to_liftout_position(microscope)

settings.image.hfw = 400e-6
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")

plt.show()


In [ ]:
microscope.move_manipulator_corrected(dx=10e-6, dy=10e-6, beam_type=BeamType.ELECTRON)

settings.image.hfw = 400e-6
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")

plt.show()

microscope.move_manipulator_corrected(dx=10e-6, dy=10e-6, beam_type=BeamType.ION)

settings.image.hfw = 400e-6
eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")

plt.show()

In [ ]:
# chamber camera
image = microscope.acquire_chamber_image()

plt.imshow(image.data, cmap="gray")
plt.show()


In [ ]:
# waffle dry run - full

# serial liftout - prep, mill trench, liftout

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, patterning


In [ ]:

microscope, settings = utils.setup_session(protocol_path="../../autolamella/autolamella/protocol/protocol-serial-liftout.yaml")

from fibsem.structures import BeamType
settings.image.hfw = 80e-6
settings.image.beam_type = BeamType.ION
image = acquire.acquire_image(microscope, settings.image)

stages = patterning.get_milling_stages("prepare-copper-release", settings.protocol["milling"])


from fibsem.ui.utils import _draw_milling_stages_on_image

# plot
fig = _draw_milling_stages_on_image(image, stages)


### Archive Models


In [ ]:
from fibsem.segmentation.model import load_model


import glob, os

from fibsem.structures import FibsemImage
from fibsem.detection import detection

# image = FibsemImage.load("../fibsem/detection/test_image.tif")

model = load_model("archive/autoliftout-latest.pt")



PATH = "/home/patrick/github/data/autolamella-paper/model-development/train/liftout/test"

filenames = glob.glob(os.path.join(PATH, "*.tif"))

for fname in filenames[:10]:
    image = FibsemImage.load(fname)
    features = [detection.NeedleTip(), detection.LamellaRightEdge()]
    det = detection.detect_features(image, model, features)
    detection.plot_detection(det)



#### Serial Liftout examples

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem.segmentation.model import load_model
from fibsem.segmentation.utils import plot_segmentations
import glob, os
import matplotlib.pyplot as plt

model = load_model("autolamella-serial-liftout-20240107.pt")

filenames = ["00065", "00089", "00146", 
             "03691", "03870", "07670", 
             "06477", "08041", "08229"]


SAVE_PATH = "/home/patrick/github/data/autolamella-paper/model-development/eval/serial-liftout-v1/best-v1"

for fname in filenames:
    print(fname)
    image = FibsemImage.load(f"/home/patrick/github/data/autolamella-paper/model-development/train/serial-liftout/test/{fname}.tif")

    mask = model.inference(image.data, rgb=False)[0]    
    

    fig = plot_segmentations(image, mask)

    # save figure, dpi=300, no axes
    fig.savefig(os.path.join(SAVE_PATH, f"{fname}.png"), dpi=300, bbox_inches="tight", pad_inches=0)
    plt.close(fig)



In [ ]:
import tifffile as tff
import os

input_path = "/home/patrick/github/data/mmtest/"
output_path = os.path.join(input_path, "sliced")
os.makedirs(output_path, exist_ok=True)

# read full image
filename = os.path.join(input_path, "MM001.tif")
image = tff.imread(filename)
print(image.shape)
for i in range(image.shape[0]):

    basename = os.path.basename(filename).replace(".tif", f"_{i:04d}.tif")
    print(f"Saving slice {i} as {basename}.")

    # save as tiff
    tff.imwrite(os.path.join(output_path, basename), image[i])



### MMTEST


In [9]:
PATH = "/home/patrick/github/data/mmtest/raw_images/labels"

import os, glob

filenames = glob.glob(os.path.join(PATH, "**.tif"))  


import tifffile as tff

for fname in filenames:
    image = tff.imread(fname)
    print(image.shape, image.dtype)


    # convert to uint8
    # image = image.astype("uint8")
    # tff.imwrite(fname, image)

(2048, 2048) uint8
(2048, 2048) uint8
(2048, 2048) uint8
(2048, 2048) uint8
(2048, 2048) uint8
